# Цербер: пересечение между сообществами

Автор: Шмаков Сергей
* http://freesmm.ru
* https://vk.com/smmblog
* https://vk.com/freesmm - сборник бесплатных скриптов для SMM
* https://youtube.com/smmblog

* **Вопросы по скрипту/доработки** : https://new.vk.com/topic-41212221_34161499

> Хотите научиться самостоятельно делать такие скрипты? Проходите бесплатный курс, который я составил и будем всем счастье! http://freesmm.tilda.ws/page241764.html

Задача:
> получить список ID пользователей, которые состоят сразу в 3-х сообществах.

Сравнивать будем:
* https://vk.com/smmblog
* https://vk.com/bizness_online
* https://vk.com/cerebro_vk

In [ ]:
Vm8WB2Uy_v0

Получить токен: https://oauth.vk.com/authorize?client_id=5453402&display=page&redirect_uri=http://localhost&scope=&response_type=token&v=5.53

In [24]:
token = '' #вставить в ковычки токен

In [1]:
import requests
import time as t 
import csv
from datetime import datetime, date, time, timedelta
import random

Функция для сбора ID из сообщества
взята из готового скрипта - https://github.com/kepatopoc/vk-scripts/blob/master/get_club_id.ipynb

In [6]:
def get_members_list_id(owner_id):    
    members_list =  [] #изначально пустой список участников

    #первый запрос на 25000, чтобы получить первые 25000 и количество участников группы
    r = requests.post('https://api.vk.com/method/execute.Shmakov_getClub_members?group_id='+
                      str(owner_id)+'&offset='+str(0)+'&count='+str(25000)+'&access_token='+token).json()['response']
    members_count = r[0] #количество участников
    
    print('В сообществе', owner_id, 'участников :',members_count)
    
    members_list.extend(r[1]) #вносим первые 25000 ID 
    
    if members_count > 25000:
        print('В сообществе',owner_id,'больше 25000 участников. Цербер запускает цикл')
        for offset in range(25000, members_count, 25000):
            count = offset + 25000

            r = requests.post('https://api.vk.com/method/execute.Shmakov_getClub_members?group_id='+
                             str(owner_id)+'&offset='+str(offset)+'&count='+str(count)+'&access_token='+token).json()['response']

            members_list.extend(r[1]) #вносим все последующие ID пачками по 25000 ID


            #t.sleep(.35) #задержки между запросом --- ВАЖНО: если будут возникать проблемы - расскоментировать 
        print('Цербер закончил сбор ID для')
    else:
        print('В сообществе меньше 25000 участников. Цербер закончил сбор ID для')

    return members_list

Запускаем функцию для каждого сообщества.

**%time** - для вывода времени, за которое выполняется функция

In [11]:
%time smmblog = get_members_list_id(38369814)
%time cerebro = get_members_list_id(73662138)
%time imaya = get_members_list_id(40018862)

В сообществе 38369814 участников : 28945
В сообществе 38369814 больше 25000 участников. Цербер запускает цикл
Цербер закончил сбор ID для
Wall time: 1.32 s
В сообществе 73662138 участников : 136479
В сообществе 73662138 больше 25000 участников. Цербер запускает цикл
Цербер закончил сбор ID для
Wall time: 4.92 s
В сообществе 40018862 участников : 104536
В сообществе 40018862 больше 25000 участников. Цербер запускает цикл
Цербер закончил сбор ID для
Wall time: 3.63 s


выводим на экран количество собранных ID

In [13]:
count = len(smmblog + cerebro + imaya)
count #269 000 за 10 сек. о как.

269960

Сколько участников состоит только в одном из 3х сообществ.

In [18]:
uniq_3 = set(smmblog) | set(cerebro) | set(imaya)
len(groups_3)

225804

![alt](https://api.monosnap.com/rpc/file/download?id=Sr0Wjle2cKKP4FYI5qy7F9hO87R1K5)<br>
Получил уникальные, то есть без каких-либо пересечений. 
Получается что всего 44000 сидит в трёх сообществах.


![alt](https://api.monosnap.com/rpc/file/download?id=kN2MR3ZGLZxLvq35hwvMGn9KZwN6uS)<br>


In [21]:
groups_3 = set(smmblog) & set(cerebro) & set(imaya)
len(groups_3)

6499

![alt](https://api.monosnap.com/rpc/file/download?id=GXusDSTx27xXJjjajYOgLwiQ44M8H3)<br>

6499 состоят сразу в трёх сообществах. Это значение без проблем можем записать в csv

## Запись данных в csv

In [23]:
with open('3 группы.csv', 'w', newline='') as csvfile:
            datawriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
            datawriter.writerow(groups_3)

Всё. файл можно загружать в ретаргетинг.

Таким образом из нескольких частей был составлен простой скрипт по поиску тех, кто находится в нескольких сообщества. Заняло это неболее 10 минут при наличии готовых компоментов и понимании что делать.